In [195]:
import pandas as pd
from scipy.optimize import linprog

#

In [196]:
df = pd.read_csv(r'DEA.csv')

input_cols_300 = [
    "Critical Power (kW)",
    'Mean Energy Consumption (300 Output Tokens)',
    'PUE',
    'WUE (Total)',
    'CIF'
]
input_cols_1000 = [
    "Critical Power (kW)",
    'Mean Energy Consumption (1000 Output Tokens)',
    'PUE',
    'WUE (Total)',
    'CIF'
]
input_cols_1500 = [
    "Critical Power (kW)",
    'Mean Energy Consumption (1500 Output Tokens)',
    'PUE',
    'WUE (Total)',
    'CIF'
]
output_cols = [
    'Score',

]



In [197]:
df.columns=df.iloc[0,:]
df=df.iloc[1:]
df.head()

,Model,Organization,API Provider,GPUs Used,System (DGX/HGX),Critical Power (kW),Idle Power,Utilization Rate (min),Utilization Rate (max),PUE,...,Score,Min Mean (300 Output Tokens),Max Mean (300 Output Tokens),Min Mean (1k Output Tokens),Max Mean (1k Output Tokens),Min Mean (1.5k Output Tokens),Max Mean (1.5k Output Tokens),Mean Speed (300 Output Tokens),Mean Speed (1000 Output Tokens),Mean Speed (1500 Output Tokens)
1,GPT-4.1,OpenAI,OpenAI,Nvidia H200/H100,HGX H200/H100,10.20,1.84,20.00%,40.00%,1.12,...,53,0.005090546,0.007517435,0.0139334897,0.0205761998,0.02346917372,0.03465796584,4.663266733,12.76396966,21.4992675
2,GPT-4.1 mini,OpenAI,OpenAI,Nvidia H200/H100,HGX H200/H100,10.20,1.84,10.00%,20.00%,1.12,...,53,0.003227037,0.004237026,0.0065009482,0.0085355961,0.01220046635,0.01601893291,3.881387283,12.88226813,14.67436936
3,GPT-4.1 nano,OpenAI,OpenAI,Nvidia H200/H100,HGX H200/H100,10.20,1.84,10.00%,20.00%,1.12,...,41,0.001460420,0.001917498,0.003850105,0.0050550997,0.00645503627,0.0084753148,1.756551086,6.22497426,7.763931624
4,o4-mini (high),OpenAI,OpenAI,Nvidia H200/H100,HGX H200/H100,10.20,1.84,10.00%,20.00%,1.12,...,70,0.022370860,0.029372428,0.0386583758,0.0507575621,0.04346265564,0.05706547153,26.90702637,25.5639879,52.27562982
5,GPT-4.5,OpenAI,OpenAI,Nvidia H200/H100,HGX H200/H100,10.20,1.84,20.00%,40.00%,1.12,...,51,0.037276864,0.055048393,0.1136651595,0.1678543634,0.1690884475,0.2497003818,34.14799717,68.59825512,154.8958565


In [198]:
df.dropna( inplace=True)

In [199]:
df.loc[df['Mean Energy Consumption (1500 Output Tokens)'] == '#DIV/0!', 'Mean Energy Consumption (1500 Output Tokens)'] = 0.0

In [230]:
# 3. Extract matrices
X_300 = df[input_cols_300].astype(float).values  # shape: (n_dmus, n_inputs)
X_1000 = df[input_cols_1000].astype(float).values  # shape: (n_dmus, n_inputs)
X_1500 = df[input_cols_1500].astype(float).values  # shape: (n_dmus, n_inputs)

Y = df[output_cols].astype(float).values  # shape: (n_dmus, n_outputs)

n_dmus, n_inputs = X_300.shape


_, n_outputs = Y.shape

eff_scores = []





In [235]:
Y

array([[53.],
       [53.],
       [41.],
       [70.],
       [51.],
       [70.],
       [66.],
       [63.],
       [62.],
       [54.],
       [50.],
       [36.],
       [60.],
       [53.],
       [57.],
       [48.],
       [44.],
       [35.],
       [24.],
       [35.],
       [40.],
       [10.],
       [20.],
       [33.],
       [41.]])

In [201]:
for i in input_cols_300:
    for j in output_cols:
        print(f'{i},{j}":", {df[i].corr(df[j])}')

Critical Power (kW),Score":", 0.14529854572644046
Mean Energy Consumption (300 Output Tokens),Score":", 0.37818223765563275
PUE,Score":", 0.025245302006388214
WUE (Total),Score":", 0.22572301211813717
CIF,Score":", 0.11960806861554564


In [202]:
X_dict = {
    '300': X_300,
    '1000': X_1000,
    '1500': X_1500
}

In [211]:

for i in X_dict:
    eff_scores = []

    for j in range(n_dmus):
        # Decision vars: v = [theta, lambda_0, ..., lambda_{n_dmus-1}]
        c = [1.0] + [0.0]*n_dmus  # minimize theta

        # Build inequality constraints A_ub * v <= b_ub
        A_ub = []
        b_ub = []

        # Input constraints: Σ λ_i * X[i,k] ≤ θ * X[j,k]
        for k in range(n_inputs):
            row = [-X_dict[i][j,k]] + list(X_dict[i][:,k])
            A_ub.append(row)
            b_ub.append(0.0)

        # Output constraints: Σ λ_i * Y[i,r] ≥ Y[j,r]
        # ↔ -Σ λ_i * Y[i,r] ≤ -Y[j,r]
        for r in range(n_outputs):
            row = [0.0] + list(-Y[:,r])
            A_ub.append(row)
            b_ub.append(-Y[j,r])

        # **No equality constraint for CCR** (drop Σ λ_i = 1)

        # Bounds: θ ≥ 0; λ_i ≥ 0
        bounds = [(0, None)] * (1 + n_dmus)

        # Solve the LP **without** A_eq/b_eq
        res = linprog(
            c=c,
            A_ub=A_ub,
            b_ub=b_ub,
            bounds=bounds,
            method='highs'
        )

        eff_scores.append(res.x[0] if res.success else None)


    df[f'DEA_Efficiency_CCR_{i}'] = eff_scores
    print(df[f'DEA_Efficiency_CCR_{i}'])
    print(f"finished {i}")


1     0.886979
2     1.000000
3     1.000000
4     1.000000
5     0.728571
6     1.000000
7     0.970916
8     1.000000
9     0.885714
10    0.932825
11    1.000000
12    1.000000
15    0.857143
16    0.764427
17    1.000000
18    0.714542
19    0.719108
20    0.574121
23    0.473470
24    0.553192
25    0.632470
26    0.208969
27    0.373230
29    0.542325
30    0.831487
Name: DEA_Efficiency_CCR_300, dtype: float64
finished 300
1     0.886478
2     1.000000
3     1.000000
4     1.000000
5     0.728571
6     1.000000
7     0.942857
8     0.900000
9     0.885714
10    0.771429
11    0.982679
12    1.000000
15    0.857143
16    0.757143
17    0.843700
18    0.710484
19    0.651277
20    0.518061
23    0.396414
24    0.518061
25    0.592070
26    0.148018
27    0.395750
29    0.589034
30    0.701583
Name: DEA_Efficiency_CCR_1000, dtype: float64
finished 1000
1     0.829328
2     1.000000
3     1.000000
4     1.000000
5     0.728571
6     1.000000
7     0.981076
8     1.000000
9     0.8857

In [212]:
df[["Model", "DEA_Efficiency_CCR_300", "DEA_Efficiency_CCR_1000", "DEA_Efficiency_CCR_1500"]]


,Model,DEA_Efficiency_CCR_300,DEA_Efficiency_CCR_1000,DEA_Efficiency_CCR_1500
1,GPT-4.1,0.886979,0.886478,0.829328
2,GPT-4.1 mini,1.000000,1.000000,1.000000
3,GPT-4.1 nano,1.000000,1.000000,1.000000
4,o4-mini (high),1.000000,1.000000,1.000000
5,GPT-4.5,0.728571,0.728571,0.728571
6,o3,1.000000,1.000000,1.000000
7,o3-mini (high),0.970916,0.942857,0.981076
8,o3-mini,1.000000,0.900000,1.000000
9,o1,0.885714,0.885714,0.885714
10,o1-mini,0.932825,0.771429,0.834024


In [214]:
for i in X_dict:
    eff_scores = []
    for j in range(n_dmus):        # Decision vars: v = [theta, lambda_0, lambda_1, ..., lambda_{n_dmus-1}]
        c = [1.0] + [0.0]*n_dmus  # objective: minimize theta
        
        # Inequality constraints A_ub * v <= b_ub
        A_ub = []
        b_ub = []
        
        # Input constraints: sum_i λ_i * X[i,k] - θ * X[j,k] <= 0
        for k in range(n_inputs):
            row = [-X_dict[i][j,k]] + list(X_dict[i][:,k])
            A_ub.append(row)
            b_ub.append(0.0)
            
        # Output constraints: -sum_i λ_i * Y[i,r] <= -Y[j,r]
        for r in range(n_outputs):
            row = [0.0] + list(-Y[:,r])
            A_ub.append(row)
            b_ub.append(-Y[j,r])
        
        # Equality constraints A_eq * v = b_eq
        # VRS constraint: sum_i λ_i = 1
        A_eq = [[0.0] + [1.0]*n_dmus]
        b_eq = [1.0]
        
        # Bounds for all variables: theta >= 0, lambda_i >= 0
        bounds = [(0, None)] * (1 + n_dmus)
        
        # Solve LP
        res = linprog(
            c=c,
            A_ub=A_ub,
            b_ub=b_ub,
            A_eq=A_eq,
            b_eq=b_eq,
            bounds=bounds,
            method='highs'
        )
        
        eff_scores.append(res.x[0] if res.success else None)

    # 5. Add results back to DataFrame
    df[f'DEA_Efficiency_BCC_{i}'] = eff_scores
    print(df[f'DEA_Efficiency_BCC_{i}'])
    print(f"finished {i}")

1     1.00000
2     1.00000
3     1.00000
4     1.00000
5     1.00000
6     1.00000
7     1.00000
8     1.00000
9     1.00000
10    1.00000
11    1.00000
12    1.00000
15    0.89331
16    0.88189
17    1.00000
18    1.00000
19    1.00000
20    1.00000
23    1.00000
24    1.00000
25    1.00000
26    1.00000
27    1.00000
29    1.00000
30    1.00000
Name: DEA_Efficiency_BCC_300, dtype: float64
finished 300
1     1.00000
2     1.00000
3     1.00000
4     1.00000
5     1.00000
6     1.00000
7     1.00000
8     1.00000
9     1.00000
10    1.00000
11    1.00000
12    1.00000
15    0.89331
16    0.88189
17    1.00000
18    1.00000
19    1.00000
20    1.00000
23    1.00000
24    1.00000
25    1.00000
26    1.00000
27    1.00000
29    1.00000
30    1.00000
Name: DEA_Efficiency_BCC_1000, dtype: float64
finished 1000
1     1.00000
2     1.00000
3     1.00000
4     1.00000
5     1.00000
6     1.00000
7     1.00000
8     1.00000
9     1.00000
10    1.00000
11    1.00000
12    1.00000
15    0.89331


In [215]:
df[["Model", "DEA_Efficiency_BCC_300", "DEA_Efficiency_BCC_1000", "DEA_Efficiency_BCC_1500"]]

,Model,DEA_Efficiency_BCC_300,DEA_Efficiency_BCC_1000,DEA_Efficiency_BCC_1500
1,GPT-4.1,1.00000,1.00000,1.00000
2,GPT-4.1 mini,1.00000,1.00000,1.00000
3,GPT-4.1 nano,1.00000,1.00000,1.00000
4,o4-mini (high),1.00000,1.00000,1.00000
5,GPT-4.5,1.00000,1.00000,1.00000
6,o3,1.00000,1.00000,1.00000
7,o3-mini (high),1.00000,1.00000,1.00000
8,o3-mini,1.00000,1.00000,1.00000
9,o1,1.00000,1.00000,1.00000
10,o1-mini,1.00000,1.00000,1.00000


In [216]:
df[["Model", "DEA_Efficiency_CCR_300", "DEA_Efficiency_CCR_1000", "DEA_Efficiency_CCR_1500"]]


,Model,DEA_Efficiency_CCR_300,DEA_Efficiency_CCR_1000,DEA_Efficiency_CCR_1500
1,GPT-4.1,0.886979,0.886478,0.829328
2,GPT-4.1 mini,1.000000,1.000000,1.000000
3,GPT-4.1 nano,1.000000,1.000000,1.000000
4,o4-mini (high),1.000000,1.000000,1.000000
5,GPT-4.5,0.728571,0.728571,0.728571
6,o3,1.000000,1.000000,1.000000
7,o3-mini (high),0.970916,0.942857,0.981076
8,o3-mini,1.000000,0.900000,1.000000
9,o1,0.885714,0.885714,0.885714
10,o1-mini,0.932825,0.771429,0.834024


In [221]:
c = [-Y[k]] + [0.0]*n_inputs
c

[array([-53.]), 0.0, 0.0, 0.0, 0.0, 0.0]

In [238]:

for i in X_dict:


    u = np.zeros(n_dmus)             # one output weight per evaluator
    v = np.zeros((n_dmus, n_inputs)) # input weights per evaluator

    # 1) Solve the multiplier DEA for each evaluator DMU k
    for k in range(n_dmus):
        # Decision vars: [u_k, v_k0, ..., v_k(m-1)]
        # Objective: maximize u_k * Y[k] -> minimize -u_k * Y[k]
        c = [-Y.flatten()[k]] + [0.0]*n_inputs

        # Constraints:
        #   for each DMU j: u_k*Y[j] - sum_i v_ki * X[j,i] <= 0
        A_ub, b_ub = [], []
        for j in range(n_dmus):
            A_ub.append([ Y.flatten()[j] ] + list(-X_dict[i][j]))
            b_ub.append(0.0)

        # Normalization constraint: sum_i v_ki * X[k,i] = 1
        A_eq = [[0.0] + list(X[k])]
        b_eq = [1.0]

        # Bounds: all weights >= ε to avoid trivial zeros
        eps = 1e-6
        bounds = [(eps, None)] * (1 + n_inputs)

        res = linprog(
            c=c,
            A_ub=A_ub, b_ub=b_ub,
            A_eq=A_eq, b_eq=b_eq,
            bounds=bounds,
            method='highs'
        )

        if not res.success:
            raise RuntimeError(f"DEA multiplier LP failed for DMU {k}")

        # store the weights
        u[k]        = res.x[0]
        v[k, :]     = res.x[1:]

    # 2) Build the cross-efficiency matrix CE[k,j]
    CE = np.zeros((n_dmus, n_dmus))
    for k in range(n_dmus):
        for j in range(n_dmus):
            CE[k, j] = (u[k] * Y[j]) / (v[k].dot(X[j]))

    # 3) Average across rows to get each DMU’s average cross-efficiency
    avg_CE = CE.mean(axis=0)   # shape: (n_dmus,)

    # 4) Attach back to your DataFrame
    df[f'Cross_Efficiency_Avg_{i}'] = avg_CE

    # 5) Inspect the ranking


/var/folders/k0/5kjb5xhn5hz5qd2_5ccp61th0000gn/T/ipykernel_5734/3275266998.py:47: DeprecationWarning:

Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)

/var/folders/k0/5kjb5xhn5hz5qd2_5ccp61th0000gn/T/ipykernel_5734/3275266998.py:47: DeprecationWarning:

Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)

/var/folders/k0/5kjb5xhn5hz5qd2_5ccp61th0000gn/T/ipykernel_5734/3275266998.py:47: DeprecationWarning:

Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)



In [239]:
df[["Model", "Cross_Efficiency_Avg_300", "Cross_Efficiency_Avg_1000", "Cross_Efficiency_Avg_1500"]]

,Model,Cross_Efficiency_Avg_300,Cross_Efficiency_Avg_1000,Cross_Efficiency_Avg_1500
1,GPT-4.1,0.830832,1.115044,1.307333
2,GPT-4.1 mini,0.899074,1.502511,1.817361
3,GPT-4.1 nano,0.755339,1.975468,2.504889
4,o4-mini (high),0.802649,0.732942,0.805404
5,GPT-4.5,0.496246,0.409312,0.431229
6,o3,0.672133,0.550964,0.577806
7,o3-mini (high),0.803332,0.770277,0.855610
8,o3-mini,0.956713,1.205762,1.400110
9,o1,0.679652,0.600789,0.654086
10,o1-mini,0.863463,1.212339,1.431080


In [241]:
df["Cross_Efficiency_Avg_Total"] = df[["Cross_Efficiency_Avg_300", "Cross_Efficiency_Avg_1000", "Cross_Efficiency_Avg_1500"]].mean(axis=1)

In [242]:
df[["Model", "Cross_Efficiency_Avg_Total"]]

,Model,Cross_Efficiency_Avg_Total
1,GPT-4.1,1.084403
2,GPT-4.1 mini,1.406315
3,GPT-4.1 nano,1.745232
4,o4-mini (high),0.780332
5,GPT-4.5,0.445595
6,o3,0.600301
7,o3-mini (high),0.809740
8,o3-mini,1.187528
9,o1,0.644843
10,o1-mini,1.168960


In [243]:
df["DEA_Efficiency_CCR_Total"] = df[["DEA_Efficiency_CCR_300", "DEA_Efficiency_CCR_1000", "DEA_Efficiency_CCR_1500"]].mean(axis=1)
df[["Model", "DEA_Efficiency_CCR_Total"]]

,Model,DEA_Efficiency_CCR_Total
1,GPT-4.1,0.867595
2,GPT-4.1 mini,1.000000
3,GPT-4.1 nano,1.000000
4,o4-mini (high),1.000000
5,GPT-4.5,0.728571
6,o3,1.000000
7,o3-mini (high),0.964950
8,o3-mini,0.966667
9,o1,0.885714
10,o1-mini,0.846093


In [244]:
df["DEA_Efficiency_BCC_Total"] = df[["DEA_Efficiency_BCC_300", "DEA_Efficiency_BCC_1000", "DEA_Efficiency_BCC_1500"]].mean(axis=1)
df[["Model", "DEA_Efficiency_BCC_Total"]]

,Model,DEA_Efficiency_BCC_Total
1,GPT-4.1,1.00000
2,GPT-4.1 mini,1.00000
3,GPT-4.1 nano,1.00000
4,o4-mini (high),1.00000
5,GPT-4.5,1.00000
6,o3,1.00000
7,o3-mini (high),1.00000
8,o3-mini,1.00000
9,o1,1.00000
10,o1-mini,1.00000


In [245]:
df[["Model", "DEA_Efficiency_CCR_Total", "DEA_Efficiency_BCC_Total","Cross_Efficiency_Avg_Total"]]

,Model,DEA_Efficiency_CCR_Total,DEA_Efficiency_BCC_Total,Cross_Efficiency_Avg_Total
1,GPT-4.1,0.867595,1.00000,1.084403
2,GPT-4.1 mini,1.000000,1.00000,1.406315
3,GPT-4.1 nano,1.000000,1.00000,1.745232
4,o4-mini (high),1.000000,1.00000,0.780332
5,GPT-4.5,0.728571,1.00000,0.445595
6,o3,1.000000,1.00000,0.600301
7,o3-mini (high),0.964950,1.00000,0.809740
8,o3-mini,0.966667,1.00000,1.187528
9,o1,0.885714,1.00000,0.644843
10,o1-mini,0.846093,1.00000,1.168960


In [251]:
for i in list(["DEA_Efficiency_CCR_Total", "DEA_Efficiency_BCC_Total","Cross_Efficiency_Avg_Total"]):
    for j in list(["DEA_Efficiency_CCR_Total", "DEA_Efficiency_BCC_Total","Cross_Efficiency_Avg_Total"]):
        if i != j:
            print(f"{i} vs {j}:", df[i].corr(df[j]))

DEA_Efficiency_CCR_Total vs DEA_Efficiency_BCC_Total: -0.05416353758745797
DEA_Efficiency_CCR_Total vs Cross_Efficiency_Avg_Total: 0.588972319873188
DEA_Efficiency_BCC_Total vs DEA_Efficiency_CCR_Total: -0.05416353758745797
DEA_Efficiency_BCC_Total vs Cross_Efficiency_Avg_Total: 0.3377673209888858
Cross_Efficiency_Avg_Total vs DEA_Efficiency_CCR_Total: 0.588972319873188
Cross_Efficiency_Avg_Total vs DEA_Efficiency_BCC_Total: 0.33776732098888584


In [209]:


# 2. Specify columns
x_col = 'Mean Energy Consumption (300 Output Tokens)'  # Input
y_col = 'Score'                                        # Output
eff_col = 'DEA_Efficiency_CCR'
model_col = 'Model'

# 3. Identify efficient frontier DMUs and sort by input
df_eff = df.loc[df[eff_col].round(3) == 1.0].sort_values(x_col)

# 4. Create the DEA scatter + frontier plot
fig = go.Figure()

# Plot all DMUs
fig.add_trace(go.Scatter(
    x=df[x_col].astype(float),
    y=df[ y_col].astype(float),
    mode='markers+text',
    text=df[ model_col],
    textposition='top center',
    textfont=dict(color='black', size=7),
    marker=dict(color='lightgrey', size=8),
    name='All DMUs',
    hovertemplate="<b>%{text}</b><br>" +
                  f"{x_col}: "+"%{x}<br>" +
                  f"{y_col}: "+"%{y}<extra></extra>"
))

# Plot efficient frontier
fig.add_trace(go.Scatter(
    x=df_eff[x_col],
    y=df_eff[y_col].astype(float),
    mode='lines+markers+text',
    textposition='bottom right',
    line=dict(color='blue', width=2),
    marker=dict(color='blue', size=10),
    name='Efficient Frontier',
    hovertemplate="<b>%{text}</b><br>" +
                  f"{x_col}: "+"%{x}<br>" +
                  f"{y_col}: "+"%{y}<extra></extra>"
))

# 5. Style the layout
fig.update_layout(
    title="CCR DEA Plot",
    xaxis_title="Mean Energy Consumption (300 Tokens)",
    yaxis_title="Benchmark Score",
    xaxis=dict(tickmode='auto'),
    margin=dict(l=60, r=20, t=60, b=60)
)

# 6. Show the plot
fig.show()


KeyError: 'DEA_Efficiency_CCR'